In [ ]:
import pandas as pd
from lxml import etree
import unicodedata
import plotly.express as px

In [ ]:
tree = etree.parse('Mittelalter.xml')
root = tree.getroot()                
ns = {'marc': 'http://www.loc.gov/MARC21/slim'} 
records = root.findall('.//marc:record', namespaces=ns)
print("Gefundene Records:", len(records))

In [ ]:
# Funktion zum Extrahieren von Datensätzen
def parse_record(record):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    
    def extract_text(xpath_query):
        fields = record.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in fields if field.text)
        return "unknown"

    idn = extract_text("marc:controlfield[@tag='001']")
    creator = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    creator_rela = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='e']")
    title = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    added_creator = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='a']")
    added_creator_rela = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='e']")
    year = extract_text("marc:controlfield[@tag='008']")
    
    return {
        "idn": idn,
        "creator": creator,
        "creator_rela": creator_rela,
        "title": title,
        "added_creator": added_creator,
        "added_creator_rela": added_creator_rela,
        "year": year[7:11]
    }

In [ ]:
# Übergabe der einzelnen Datensätze an die Funktion "parse_record":
result = [parse_record(record) for record in records]
df = pd.DataFrame(result)
df

Die einzelnen Einträge unter additional_author und added_rela sind jeweils durch ein Semikolon getrennt - da es im Eintrag zu Franz Siegfried Müller allerdings zwei Einträge der Relation (zwei subfields $e) für beide Funktionen als Verfasser sowie als Herausgeber gab, sind die Einträge hier nicht mehr eindeutig zuordnenbar (2 Einträge zu additional_authors vs. 3 added_rela-Einträge).

Für eine genauere Analyse der einzelnen Personen inklusive der Beziehungen zum Werk ist es also nötig, die Funktion anzupassen und die einzelnen Unterfelder in Abhängigkeit ihrer "datafields" zu extrahieren und die Abhängigkeiten entsprechend abzubilden. Dazu wird die Funktion von eben etwas verändert und erweitert:


In [ ]:
# Erweiterte Funktion zum Extrahieren der Inhalte: 
def parse_record_advanced(record):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}

    def extract_text(xpath_query):
        element = xpath_query.split("/")
        parentpath = xpath_query.split("/")[0]
        if len(element) == 1:
            fields = record.xpath(xpath_query, namespaces=ns)
            if fields:
                return "; ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in fields if field.text)
            return "unknown"
        else:
            childpath = xpath_query.split("/")[1]
            parentfields = record.xpath(parentpath, namespaces=ns)
            result = []
            
            for parent in parentfields:
                subfields = parent.xpath(childpath, namespaces=ns)
                if subfields:
                    text = " / ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in subfields if field.text)
                else:
                    text = ""
                result.append(text)
    
            if len(result) == 0:
                return None
            elif len(result) == 1:
                return result[0]
            else:
                return result
                

    # Extract fields
    idn = extract_text("marc:controlfield[@tag='001']")
    creator = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    creator_rela = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='e']")
    title = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    added_creator = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='a']")
    added_creator_rela = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='e']")
    year = extract_text("marc:controlfield[@tag='008']")
  
    return {
        "idn": idn,
        "creator": creator,
        "creator_rela": creator_rela,
        "title": title,
        "added_creator": added_creator,
        "added_creator_rela": added_creator_rela,
        "year": year[7:11]
    }

In [ ]:
result_new = [parse_record_advanced(record) for record in records]
df_new = pd.DataFrame(result_new)
df_new

#### Unwandeln des Dataframes für die weitere Analyse

Das aktuelle Dataframe verfügt nun über jeweils 2 Spalten, in denen Personen gelistet werden, sowie zwei zugehörige Spalten, die deren Relation zum Werk beschreiben. Um im weiteren Verlauf alle Personen gleichermaßen listen und analyiseren zu können, muss das Dataframe nun noch etwas angepasst werden.

Ziel ist es, nur noch eine Spalte "person" mit einer zugehörigen Spalte "person_rela" zu erzeugen und die Daten aus author, author_rela, additional_author und added_rela entsprechend in diese Spalten zu überführen. Außerdem soll die Beziehung der jeweiligen Person zum Werk erhalten bleiben. Das neue Dataframe sollte also in etwas folgendermaßen aussehen:

| idn        | titel	                                         | person                         | person_rela	          | jahr | 
|------------|---------------------------------------------------|--------------------------------|-----------------------|------| 
| 1343800412 | Asynchrone Zeitraster: Bildzyklen im Kölner D...  | Freigang, Christian            | Verfasser             | 2024 |
| 1321969694 | Changes of Monarchical Rule in the Late Middle... | Jaros, Sven                    | Herausgeber           | 2024 |
| 1321969694 | Changes of Monarchical Rule in the Late Middle... | Böhme, Eric 	                  | Herausgeber           | 2024 |
| 1321969694 | Changes of Monarchical Rule in the Late Middle... | Jaros, Marie Ulrik..	          | Herausgeber           | 2024 |
| usw. | | | | |

Um dies zu erreichen sind mehrere Schritte nötig: Zunächst werden die Listen, die sich in additional_author und added_rela befinden mit Hilfe der .explode()-Funktion von Pandas "entpackt". Dies bewirkt, dass eine Spalte, die Listen oder ähnliche Strukturen enthält so dupliziert wird, dass jede Listeneinheit in eine eigene Zeile aufgeteilt wird, während die anderen Spalten unverändert bleiben. Gleichzeitig werden die Spalten author und author_rela für die gewünschte neue Struktur umbenannt:

In [ ]:
df_explode = df_new.explode(['added_creator', 'added_creator_rela'])
df_persons = df_explode.rename(columns={'creator': 'person', 'creator_rela': 'person_rela'}).copy()
df_persons

Nun wird ein neues Dataframe erstellt, welches folgendermaßen befüllt wird:

  - Kopieren jeder Zeile in das neue Dataframe
  - Prüfung, ob ein Eintrag unter additional_author vorhanden ist. Wenn ja:
    - Erneutes Kopieren der Zeile
    - Überschreiben der Einträge in person und person_rela mit den Werten aus additional_author und added_rela in der neuen Zeile



In [ ]:
# Erstellen des neuen Dataframes: 
new_rows = []

for index, row in df_persons.iterrows():
    new_rows.append(row) # Kopieren der Zeilen

    if pd.notna(row['added_creator']):
        new_row = row.copy() 
        new_row['person'] = row['added_creator'] 
        new_row['person_rela'] = row['added_creator_rela']  
        new_rows.append(new_row)  

new_df = pd.DataFrame(new_rows)
new_df.head()

Entfernen der Spalten `added_creator` und `added_creator_rela` mit anschließender Löschung doppelter Zeilen sowie von Zeilen, die keinen Eintrag unter "person" haben (da sie ursprünglich bspw. nicht über einen 100er Eintrag, sondern nur über einen 700er Eintrag verfügten):

In [ ]:
new_df = new_df.drop(columns=['added_creator', 'added_creator_rela'])
new_df.drop_duplicates(inplace=True)
new_df.dropna(subset=['person'], inplace=True)
new_df

In [ ]:
# Optional: Speichern des Dataframes als CSV
new_df.to_csv("personen.csv", encoding="utf-8")

### Analyse:

Gruppieren und Zählen der einzelnen Personen:


In [ ]:
# Häufigkeit der Autoren zählen
person_counts = new_df['person'].value_counts().head(25).reset_index()
person_counts.columns=['person', 'count']
person_counts

In [ ]:
# Balkendiagramm erstellen
fig1 = px.bar(person_counts, x='person', y='count', height=550,
             title='Häufigste Personen im Datenset', color='count', color_continuous_scale='Viridis')
fig1.show()

Suche im Dataframe nach einer bestimmten Person:

In [ ]:
new_df.loc[new_df['person'] == 'Baumgärtner, Ingrid']

Häufigkeiten der verschiedenen Rollen:

In [ ]:
# Häufigkeit von "person_rela"
rela_counts = new_df['person_rela'].value_counts().head(25).reset_index()
rela_counts.columns=['person_rela', 'count']

# Kreisdiagramm erstellen
fig2 = px.pie(rela_counts, values='count', names='person_rela', title='Verteilung der Rollen (`person_rela`) im Datenset', 
              color_discrete_sequence=px.colors.sequential.RdBu, height=500)
fig2.show()